# Network Traffic Heatmap

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo

from IPython.display import display, HTML, Markdown
from plotly.subplots import make_subplots

import nsys_display

display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pyo.init_notebook_mode()

In [ ]:
df = pd.read_parquet('analysis.parquet')
# Create unique name.
df['Name'] = df['Rank'].astype(str) + '/' + df['Device name'].astype(str) + '/' + df['GUID']
# Convert ns to s.
df['Duration'] = df['Duration'] * 1e-9

types = []

if 'IB Bytes Recv' in df or 'Eth Bytes Recv' in df:
    received_bytes = df['IB Bytes Recv'].fillna(0) if 'IB Bytes Recv' in df else 0
    received_bytes += df['Eth Bytes Recv'].fillna(0) if 'Eth Bytes Recv' in df else 0
    # Convert bytes to MBytes and rate metrics from "/ms" to "/s"
    df['MB Recv'] = (received_bytes * 1000) / (1024 * 1024)
    types.append('MB Recv')

if 'IB Bytes Sent' in df or 'Eth Bytes Sent' in df:
    sent_bytes = df['IB Bytes Sent'].fillna(0) if 'IB Bytes Sent' in df else 0
    sent_bytes += df['Eth Bytes Sent'].fillna(0) if 'Eth Bytes Sent' in df else 0
    # Convert bytes to MBytes and rate metrics from "/ms" to "/s"
    df['MB Sent'] = (sent_bytes * 1000) / (1024 * 1024)
    types.append('MB Sent')

if 'Congestion' in df:
    df['Congestion'] = df['Congestion'].fillna(0) * 1000
    types.append('Congestion')

This line graph displays the summary of data received and sent rates by all the profiled network devices:
* x axis represents the rank duration, scaling from 0 to the maximum duration across all ranks.
* y axis represents the mean value of data received and sent across all ranks.
    - The data transmitted is Ethernet and InfiniBand traffic accumulated.

In [ ]:
nsys_display.display_summary_graph(
    df,
    ['MB Recv', 'MB Sent'],
    xaxis_title="Duration (s)",
    yaxis_title="Data rate (MB/s)",
    title="Data Transmission Summary (bins=REPLACE_BIN)"
)

These heatmaps display Network devices metrics values across all NIC and IB Switch interfaces on all ranks:
* x axis represents the rank duration, scaling from 0 to the maximum duration across all ranks.
* y axis represents the set of Rank/Device name/GUID for which metrics were collected.

The heatmaps present:
* MB Recv present the rates of bytes received in MiB/s.
* MB Sent present the rates of bytes sent in MiB/s.
* Congestion presents the congestion reported by 'Send Waits' packets in ticks/s.

In [ ]:
for type in types:
    fig = make_subplots(1, 1, subplot_titles=[type], vertical_spacing=0.1)
    fig.add_trace(
        go.Heatmap(
            x=df['Duration'],
            y=df['Name'],
            z=df[type],
            showscale=False,
            zmax=df[type].max(),
            zauto=False,
            hovertemplate=('<br>'.join([
                'Duration (s): %{{x}}',
                'Name: %{{y}}',
                '{} (Value): %{{z}}'])+'<extra></extra>').format(type)),
        1, 1)
    unique_name_count = df['Name'].nunique()
    fig.update_layout(
        height=nsys_display._get_heatmap_height(unique_name_count),
        title='Network Device Metric (bins=REPLACE_BIN)'
    )
    fig.update_xaxes(title_text='Duration (s)')
    fig.update_yaxes(title_text='Rank/Device name/GUID', categoryorder="category descending", nticks=df["Name"].nunique())
    fig.update_traces({'colorbar': {'title_text': 'Value'}}, showscale=True, row=0)

    fig.show()

## Files

The table associates each rank number with the original filename. Ranks are assigned assuming that the file names include the rank with sufficient zero padding for proper sorting. Otherwise, the actual rank may differ from the assigned ID.

In [ ]:
files_df = pd.read_parquet("files.parquet")
display(files_df)